# Финальное задание

## 1 Импорт основных библиотек и объявление функций

In [151]:
%%time
%pylab inline
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import gc
import sys

from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.cluster import *
from sklearn.metrics import *
from scipy.stats import *
import folium
import geopandas as gpd
from shapely.geometry import *

import plotly
from plotly.offline import *
import plotly.plotly as py
from plotly.graph_objs import *
init_notebook_mode(connected=True)

from IPython.html.widgets import *

Populating the interactive namespace from numpy and matplotlib


Wall time: 3.95 ms


In [46]:
#функция для очистки данных
def data_prep(df):

    df = df[df['tpep_dropoff_datetime']!=df['tpep_pickup_datetime']]
    df = df[(df['passenger_count']!=0) & (df['trip_distance']!=0)]
    df = df[(df['pickup_longitude']>=reg['west'].min())]
    df = df[(df['pickup_longitude']<=reg['east'].max())]
    df = df[(df['pickup_latitude']>=reg['south'].min())]
    df = df[(df['pickup_latitude']<=reg['north'].max())]
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
    df['tpep_pickup_datetime'] = df['tpep_pickup_datetime'].apply(lambda x: x.replace(second=0))
    df['tpep_pickup_datetime'] = df['tpep_pickup_datetime'].apply(lambda x: x.replace(minute=0))
    df['tpep_dropoff_datetime'] = df['tpep_dropoff_datetime'].apply(lambda x: x.replace(second=0))
    df['tpep_dropoff_datetime'] = df['tpep_dropoff_datetime'].apply(lambda x: x.replace(minute=0))
    
    return df

In [47]:
#функция для расчета количества поездок в квартале
def data_proc(df, hours, reg):

    for i in range(hours):
        
        day_id = df['tpep_pickup_datetime'].values[0] + np.timedelta64(i, 'h')
        df_temp = df[df['tpep_pickup_datetime']==day_id]
            
        bsd = \
        binned_statistic_2d(
            x=df_temp['pickup_longitude'],
            y=df_temp['pickup_latitude'],
            values=None,
            statistic='count',
            bins=50,
            range=[[reg['west'].min(), reg['east'].max()], [reg['south'].min(), reg['north'].max()]],
            expand_binnumbers=False
        )
        
        if i%24<10:
            time = '0' + str(i%24) + ':00:00'
        else:
            time = str(i%24) + ':00:00'
            
        reg[str(day_id)[:10] + ' ' + time] = bsd.statistic.reshape(2500)

        h = i+1

        if (h%24==0 and i!=0):
            print(
                'day',
                int(h/24)
            )
                
    return reg

In [48]:
#функция с генерацией признаков для построения регрессионной модели

def df_feat(df, r):

    df['region'] = r
    df.rename(columns={r: 'cnt'}, inplace=True)
    df['hour'] = df['datetime'].dt.hour
    df['day'] = df['datetime'].dt.day
    df['dayofweek'] = df['datetime'].dt.dayofweek
    df['weekend'] = (df['datetime'].dt.weekday>=5).astype(int)
    df['cnt_rolling_mean_12'] = df['cnt'].rolling(window=12).mean()
    df['cnt_rolling_mean_24'] = df['cnt'].rolling(window=24).mean()
    df['cnt_rolling_mean_48'] = df['cnt'].rolling(window=48).mean()

    df['hour_mean'] = df.groupby(by=['hour'])['cnt'].transform('mean')
    df['day_mean'] = df.groupby(by=['day'])['cnt'].transform('mean')
    df['dayofweek_mean'] = df.groupby(by=['dayofweek'])['cnt'].transform('mean')
    df['weekend_mean'] = df.groupby(by=['weekend'])['cnt'].transform('mean')

    df['sin_value_6'] = np.sin(2*np.pi * df['datetime'].dt.hour/6)
    df['cos_value_6'] = np.cos(2*np.pi * df['datetime'].dt.hour/6)

    df['sin_value_12'] = np.sin(2*np.pi * df['datetime'].dt.hour/12)
    df['cos_value_12'] = np.cos(2*np.pi * df['datetime'].dt.hour/12)

    df['sin_value_16'] = np.sin(2*np.pi * df['datetime'].dt.hour/16)
    df['cos_value_16'] = np.cos(2*np.pi * df['datetime'].dt.hour/16)

    df['sin_value_24'] = np.sin(2*np.pi * df['datetime'].dt.hour/24)
    df['cos_value_24'] = np.cos(2*np.pi * df['datetime'].dt.hour/24)

    df['sin_value_48'] = np.sin(2*np.pi * df['datetime'].dt.hour/48)
    df['cos_value_48'] = np.cos(2*np.pi * df['datetime'].dt.hour/48)

    df['sin_value_96'] = np.sin(2*np.pi * df['datetime'].dt.hour/96)
    df['cos_value_96'] = np.cos(2*np.pi * df['datetime'].dt.hour/96)

    df['sin_value_168'] = np.sin(2*np.pi * df['datetime'].dt.hour/168)
    df['cos_value_168'] = np.cos(2*np.pi * df['datetime'].dt.hour/168)

    df.fillna(0, inplace=True)
    
    return df

## 2 Загружаем и подготавливаем данные данные поездок

In [49]:
%%time
#загружаем данные с ноября 2015 по июнь 2016

df_201511 = pd.read_csv(r'D:\Python\Coursera\6\4\yellow_tripdata_2015-11.csv')
df_201511.sort_values(by=['tpep_pickup_datetime'], inplace=True)
df_201511.reset_index(drop=True)

df_201512 = pd.read_csv(r'D:\Python\Coursera\6\4\yellow_tripdata_2015-12.csv')
df_201512.sort_values(by=['tpep_pickup_datetime'], inplace=True)
df_201512.reset_index(drop=True)

df_201601 = pd.read_csv(r'D:\Python\Coursera\6\4\yellow_tripdata_2016-01.csv')
df_201601.sort_values(by=['tpep_pickup_datetime'], inplace=True)
df_201601.reset_index(drop=True)

df_201602 = pd.read_csv(r'D:\Python\Coursera\6\4\yellow_tripdata_2016-02.csv')
df_201602.sort_values(by=['tpep_pickup_datetime'], inplace=True)
df_201602.reset_index(drop=True)

df_201603 = pd.read_csv(r'D:\Python\Coursera\6\4\yellow_tripdata_2016-03.csv')
df_201603.sort_values(by=['tpep_pickup_datetime'], inplace=True)
df_201603.reset_index(drop=True)

df_201604 = pd.read_csv(r'D:\Python\Coursera\6\4\yellow_tripdata_2016-04.csv')
df_201604.sort_values(by=['tpep_pickup_datetime'], inplace=True)
df_201604.reset_index(drop=True)

df_201605 = pd.read_csv(r'D:\Python\Coursera\6\1\yellow_tripdata_2016-05.csv')
df_201605.sort_values(by=['tpep_pickup_datetime'], inplace=True)
df_201605.reset_index(drop=True)

reg = pd.read_csv(r'D:\Python\Coursera\6\1\regions.csv', sep=';')

Wall time: 3min 33s


In [50]:
%%time

#чистим и готовим данные за каждый месяц отдельно

df_201511 = data_prep(df_201511)
df_201512 = data_prep(df_201512)
df_201601 = data_prep(df_201601)
df_201602 = data_prep(df_201602)
df_201603 = data_prep(df_201603)
df_201604 = data_prep(df_201604)
df_201605 = data_prep(df_201605)

Wall time: 19min 42s


## 3 Отрисуем кварталы Нью-Йорка, где было в мае более 5 поездок

In [51]:
%%time

reg = pd.read_csv(r'D:\Python\Coursera\6\1\regions.csv', sep=';')

bsd = \
binned_statistic_2d(
    x=df_201605['pickup_longitude'],
    y=df_201605['pickup_latitude'],
    values=None,
    statistic='count',
    bins=50,
    range=[[reg['west'].min(), reg['east'].max()], [reg['south'].min(), reg['north'].max()]],
    expand_binnumbers=False
)

reg['count'] = bsd.statistic.reshape(2500)

reg['avg'] = reg['count']/(31*24)
reg_filtered = reg[reg['avg']>=5]

Wall time: 5.03 s


In [52]:
blocks = gpd.GeoSeries(
    reg_filtered.apply(lambda x: Polygon(zip([x['west'], x['east'], x['east'], x['west']], [x['north'], x['north'], x['south'], x['south']])), axis=1),
    crs={'init': 'epsg:4326'}
)
reg_geo = gpd.GeoDataFrame(
    reg_filtered.drop(labels=['west', 'east', 'south', 'north'], axis=1),
    geometry=blocks
)

ESB = [40.748817, -73.985428]
m = folium.Map(
    location=ESB,
    zoom_start=11
)

m.choropleth(
    geo_data=reg_geo.to_json(),
    data=reg_geo,
    columns=['region', 'avg'],
    key_on='feature.properties.{}'.format('region'),
    fill_color='YlOrRd',
    fill_opacity=.6,
    line_opacity=.2
)
folium.LayerControl().add_to(m)

display(m)

## 4 Сохраняем список кварталов города, которые отрисовали выше

In [53]:
regions = reg_filtered['region'].to_list()

## 5 Собираем общий датасет с ноября 2015 по май 2016

In [54]:
%%time

reg = pd.read_csv(r'D:\Python\Coursera\6\1\regions.csv', sep=';')

reg = data_proc(df_201511, 30*24, reg)
reg = data_proc(df_201512, 31*24, reg)
reg = data_proc(df_201601, 31*24, reg)
reg = data_proc(df_201602, 28*24, reg)
reg = data_proc(df_201603, 31*24, reg)
reg = data_proc(df_201604, 30*24, reg)
reg = data_proc(df_201605, 31*24, reg)

day 1
day 2
day 3
day 4
day 5
day 6
day 7
day 8
day 9
day 10
day 11
day 12
day 13
day 14
day 15
day 16
day 17
day 18
day 19
day 20
day 21
day 22
day 23
day 24
day 25
day 26
day 27
day 28
day 29
day 30
day 1
day 2
day 3
day 4
day 5
day 6
day 7
day 8
day 9
day 10
day 11
day 12
day 13
day 14
day 15
day 16
day 17
day 18
day 19
day 20
day 21
day 22
day 23
day 24
day 25
day 26
day 27
day 28
day 29
day 30
day 31
day 1
day 2
day 3
day 4
day 5
day 6
day 7
day 8
day 9
day 10
day 11
day 12
day 13
day 14
day 15
day 16
day 17
day 18
day 19
day 20
day 21
day 22
day 23
day 24
day 25
day 26
day 27
day 28
day 29
day 30
day 31
day 1
day 2
day 3
day 4
day 5
day 6
day 7
day 8
day 9
day 10
day 11
day 12
day 13
day 14
day 15
day 16
day 17
day 18
day 19
day 20
day 21
day 22
day 23
day 24
day 25
day 26
day 27
day 28
day 1
day 2
day 3
day 4
day 5
day 6
day 7
day 8
day 9
day 10
day 11
day 12
day 13
day 14
day 15
day 16
day 17
day 18
day 19
day 20
day 21
day 22
day 23
day 24
day 25
day 26
day 27
day 28
day 29
da

In [55]:
reg = reg[reg['region'].isin(regions)]

In [56]:
df = reg.drop(labels=['region', 'west', 'east', 'south', 'north'], axis=1).T
df.set_index(pd.to_datetime(df.index), inplace=True)
df.reset_index(inplace=True)
df.rename(columns={'index': 'datetime'}, inplace=True)

In [96]:
%%time

data = pd.DataFrame()

for b in df.columns[1:]:
    
    data = pd.concat([data, df_feat(df[['datetime', b]], b)], axis=0)
    print(b)

1074
1075
1076
1124
1125
1126
1127
1128
1129
1130
1131
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1220
1221
1222
1223
1224
1226
1227
1228
1229
1230
1231
1232
1233
1234
1271
1272
1273
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1325
1326
1330
1331
1332
1333
1334
1335
1336
1337
1338
1375
1376
1377
1379
1381
1382
1383
1384
1385
1386
1387
1388
1389
1425
1430
1433
1434
1435
1436
1437
1438
1440
1441
1479
1481
1482
1529
1531
1532
1579
1629
1683
1732
1733
1782
2067
2068
2117
2118
2167
Wall time: 5min 24s


In [97]:
data['region'] = data['region']+1

In [37]:
df_full = pd.concat([df_201511, df_201512, df_201601, df_201602, df_201603, df_201604, df_201605])
del df_201511
del df_201512
del df_201601
del df_201602
del df_201603
del df_201604
del df_201605
gc.collect()

42

In [59]:
%%time

data_full = pd.DataFrame()

for r in reg['region']:
    
    print(r)
    
    west = reg[reg['region']==r]['west'].values[0]
    east = reg[reg['region']==r]['east'].values[0]
    south = reg[reg['region']==r]['south'].values[0]
    north = reg[reg['region']==r]['north'].values[0]
    
    df_temp = df_full[(df_full['pickup_longitude']>=west) & (df_full['pickup_longitude']<=east) & (df_full['pickup_latitude']>=south) & (df_full['pickup_latitude']<=north)]\
                .groupby(by=['tpep_pickup_datetime'])[['passenger_count',
                                                      'trip_distance',
                                                      'fare_amount',
                                                      'extra',
                                                      'mta_tax',
                                                      'tip_amount',
                                                      'tolls_amount',
                                                      'improvement_surcharge',
                                                      'total_amount']].mean().reset_index()
    
    df_temp['region'] = r
    
    data_full = pd.concat([data_full, df_temp])
    
data_full.rename(columns={'tpep_pickup_datetime': 'datetime'}, inplace=True)

1075
1076
1077
1125
1126
1127
1128
1129
1130
1131
1132
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1221
1222
1223
1224
1225
1227
1228
1229
1230
1231
1232
1233
1234
1235
1272
1273
1274
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1326
1327
1331
1332
1333
1334
1335
1336
1337
1338
1339
1376
1377
1378
1380
1382
1383
1384
1385
1386
1387
1388
1389
1390
1426
1431
1434
1435
1436
1437
1438
1439
1441
1442
1480
1482
1483
1530
1532
1533
1580
1630
1684
1733
1734
1783
2068
2069
2118
2119
2168
Wall time: 48.5 s


In [98]:
data = data.merge(data_full, how='left', on=['datetime', 'region'])
data.fillna(0, inplace=True)

## 6 Разбиваем данные на обучение и тест

In [61]:
X_train = data[data['datetime']<'2016-04-30 23:00:00']
X_test = data[(data['datetime']>='2016-04-30 23:00:00') & (data['datetime']<='2016-05-31 17:00:00')]

## 7 обучаем сверхслучайный лес и делаем прогноз на май 2016

In [72]:
%%time

features = X_train.columns[2:].to_list()

for h in range(1, 7):

    model = ExtraTreesRegressor(
        n_estimators=100,
        criterion='mse',
        max_depth=10,
        min_samples_split=2,
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        max_features='auto',
        max_leaf_nodes=None,
        min_impurity_decrease=0.0,
        min_impurity_split=None,
        bootstrap=True,
        oob_score=False,
        n_jobs=-1,
        random_state=1,
        verbose=0,
        warm_start=False
    )

    model.fit(
        X=X_train[features].iloc[:X_train.shape[0]-h],
        y=X_train['cnt'].shift(-h).dropna()
    )

    y_pred = model.predict(
        X=X_test[features]
    )

    date_from = pd.to_datetime('2016-04-30 23:00:00') + pd.Timedelta(hours=h)
    date_to = pd.to_datetime('2016-05-31 17:00:00') + pd.Timedelta(hours=h)
    
    err = mean_absolute_error(
        y_true=data[(data['datetime']>=date_from) & (data['datetime']<=date_to)]['cnt'],
        y_pred=y_pred
    )
    
    print(
        'HOUR:',
        h,
        'ERROR:',
        err
    )
    
    X_test['y_pred' + str(h)] = y_pred

HOUR: 1 ERROR: 25.023576342720837
HOUR: 2 ERROR: 26.76872986802496
HOUR: 3 ERROR: 27.70551219016707
HOUR: 4 ERROR: 28.708968418520843
HOUR: 5 ERROR: 28.295740514039593
HOUR: 6 ERROR: 28.982492535678876
Wall time: 1min 50s


## 8 карта с прогнозными значениями поездок за май 2016

1. карты с визуализацией реального и прогнозируемого спроса на такси в выбираемый пользователем момент времени

In [155]:
def f(x):

    columns = {
        0: 'cnt',
        1: 'y_pred1',
        2: 'y_pred2',
        3: 'y_pred3',
        4: 'y_pred4',
        5: 'y_pred5',
        6: 'y_pred6',
    }
    
    if x==0:
        print('Отображены фактические данные')
    else:
        print('Отображены данные с пронозом на', x, 'час вперед')
    
    reg_pred = reg_filtered.drop(labels=['count', 'avg'], axis=1)
    reg_pred_ = X_test.groupby(by=['region'])[['cnt', 'y_pred1', 'y_pred2', 'y_pred3', 'y_pred4', 'y_pred5', 'y_pred6']].sum().reset_index()
    reg_pred = pd.merge(reg_pred, reg_pred_, on=['region'], how='inner')

    blocks = gpd.GeoSeries(
        reg_pred.apply(lambda x: Polygon(zip([x['west'], x['east'], x['east'], x['west']], [x['north'], x['north'], x['south'], x['south']])), axis=1),
        crs={'init': 'epsg:4326'}
    )

    reg_geo = gpd.GeoDataFrame(
        reg_pred.drop(labels=['west', 'east', 'south', 'north'], axis=1),
        geometry=blocks
    )

    ESB = [40.748817, -73.985428]
    m = folium.Map(
        location=ESB,
        zoom_start=11
    )

    m.choropleth(
        geo_data=reg_geo.to_json(),
        data=reg_geo,
        columns=['region', columns[x]],
        key_on='feature.properties.{}'.format('region'),
        fill_color='YlOrRd',
        fill_opacity=.6,
        line_opacity=.2
    )
    folium.LayerControl().add_to(m)

    display(m)
    
interact(f, x=(0, 6, 1))

interactive(children=(IntSlider(value=3, description='x', max=6), Output()), _dom_classes=('widget-interact',)…

<function __main__.f(x)>

## 9 интерактивый график факт/прогноз за май 2016

2. временной ряд фактического и прогнозируемого спроса на такси в выбираемой области.

In [113]:
#отрисовал только 1 квартал, чтобы не тормозил ноутбук

#1
date_from = pd.to_datetime('2016-04-30 23:00:00') + pd.Timedelta(hours=1)
date_to = pd.to_datetime('2016-05-31 17:00:00') + pd.Timedelta(hours=1)

trace1_true = Scatter(
    x = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['datetime'],
    y = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['cnt'],
    mode='markers',
    name='Факт на 1 час вперед'
)

trace1_pred = Scatter(
    x = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['datetime'],
    y = X_test[X_test['region']==1075]['y_pred1'],
    mode='markers',
    name='Прогноз на 1 час вперед'
)

#2
date_from = pd.to_datetime('2016-04-30 23:00:00') + pd.Timedelta(hours=2)
date_to = pd.to_datetime('2016-05-31 17:00:00') + pd.Timedelta(hours=2)

trace2_true = Scatter(
    x = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['datetime'],
    y = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['cnt'],
    mode='markers',
    name='Факт на 2 часа вперед'
)

trace2_pred = Scatter(
    x = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['datetime'],
    y = X_test[X_test['region']==1075]['y_pred2'],
    mode='markers',
    name='Прогноз на 2 часа вперед'
)

#3
date_from = pd.to_datetime('2016-04-30 23:00:00') + pd.Timedelta(hours=3)
date_to = pd.to_datetime('2016-05-31 17:00:00') + pd.Timedelta(hours=3)

trace3_true = Scatter(
    x = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['datetime'],
    y = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['cnt'],
    mode='markers',
    name='Факт на 3 часа вперед'
)

trace3_pred = Scatter(
    x = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['datetime'],
    y = X_test[X_test['region']==1075]['y_pred3'],
    mode='markers',
    name='Прогноз на 3 часа вперед'
)

#4
date_from = pd.to_datetime('2016-04-30 23:00:00') + pd.Timedelta(hours=4)
date_to = pd.to_datetime('2016-05-31 17:00:00') + pd.Timedelta(hours=4)

trace4_true = Scatter(
    x = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['datetime'],
    y = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['cnt'],
    mode='markers',
    name='Факт на 4 часа вперед'
)

trace4_pred = Scatter(
    x = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['datetime'],
    y = X_test[X_test['region']==1075]['y_pred4'],
    mode='markers',
    name='Прогноз на 4 часа вперед'
)

#5
date_from = pd.to_datetime('2016-04-30 23:00:00') + pd.Timedelta(hours=5)
date_to = pd.to_datetime('2016-05-31 17:00:00') + pd.Timedelta(hours=5)

trace5_true = Scatter(
    x = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['datetime'],
    y = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['cnt'],
    mode='markers',
    name='Факт на 5 часов вперед'
)

trace5_pred = Scatter(
    x = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['datetime'],
    y = X_test[X_test['region']==1075]['y_pred5'],
    mode='markers',
    name='Прогноз на 5 часов вперед'
)

#6
date_from = pd.to_datetime('2016-04-30 23:00:00') + pd.Timedelta(hours=6)
date_to = pd.to_datetime('2016-05-31 17:00:00') + pd.Timedelta(hours=6)

trace6_true = Scatter(
    x = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['datetime'],
    y = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['cnt'],
    mode='markers',
    name='Факт на 6 часов вперед'
)

trace6_pred = Scatter(
    x = data[(data['region']==1075) & (data['datetime']>=date_from) & (data['datetime']<=date_to)]['datetime'],
    y = X_test[X_test['region']==1075]['y_pred6'],
    mode='markers',
    name='Прогноз на 6 часов вперед'
)

data_ = [
    trace1_true, trace1_pred,
    trace2_true, trace2_pred,
    trace3_true, trace3_pred,
    trace4_true, trace4_pred,
    trace5_true, trace5_pred,
    trace6_true, trace6_pred
]

updatemenus = list(
    [
        dict(
            type='buttons',
            active=0,
            buttons=list(
                [
                    dict(
                        label='+1 час',
                        method='update',
                        args = [
                            {
                                'visible': [
                                    True, True,
                                    False, False,
                                    False, False,
                                    False, False,
                                    False, False,
                                    False, False
                                ]
                            }
                        ]
                    ),
                    dict(
                        label='+2 часа',
                        method='update',
                        args = [
                            {
                                'visible': [
                                    False, False,
                                    True, True,
                                    False, False,
                                    False, False,
                                    False, False,
                                    False, False
                                ]
                            }
                        ]
                    ),
                    dict(
                        label='+3 часа',
                        method='update',
                        args = [
                            {
                                'visible': [
                                    False, False,
                                    False, False,
                                    True, True,
                                    False, False,
                                    False, False,
                                    False, False
                                ]
                            }
                        ]
                    ),
                    dict(
                        label='+4 часа',
                        method='update',
                        args = [
                            {
                                'visible': [
                                    False, False,
                                    False, False,
                                    False, False,
                                    True, True,
                                    False, False,
                                    False, False
                                ]
                            }
                        ]
                    ),
                    dict(
                        label='+5 часов',
                        method='update',
                        args = [
                            {
                                'visible': [
                                    False, False,
                                    False, False,
                                    False, False,
                                    False, False,
                                    True, True,
                                    False, False
                                ]
                            }
                        ]
                    ),
                    dict(
                        label='+6 часов',
                        method='update',
                        args = [
                            {
                                'visible': [
                                    False, False,
                                    False, False,
                                    False, False,
                                    False, False,
                                    False, False,
                                    True, True
                                ]
                            }
                        ]
                    )
                ]
            )
        )
    ]
)

layout = dict(
    title='Факт и прогноз',
    updatemenus=updatemenus
)

fig = dict(data=data_, layout=layout)

iplot(fig)

# 10 Сохраняем ноутбук

In [156]:
!jupyter nbconvert --to html Coursera_6_7.ipynb

[NbConvertApp] Converting notebook Coursera_6_7.ipynb to html
[NbConvertApp] Writing 821986 bytes to Coursera_6_7.html
